In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
#from  nulls_injector import NullsInjector
from Inject_Missing_Values import *
from Miss import *

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [5]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
df = pd.read_csv(r'C:\Users\zhossai3\Desktop\My code\OT_Fairness\Data\German Credit\German_Credit_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame

X= df.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = df.iloc[:, -1]  # Selects all rows and only the last column

# Display the DataFrames
print(X)  # Print first few rows of features
 # Print first few rows of target variable
type(X)


     checking-account  duration  credit-history  purpose  credit-amount  \
0                   1         6               0        7           1169   
1                   0        48               1        7           5951   
2                   3        12               0        4           2096   
3                   1        42               1        5           7882   
4                   1        24               2        1           4870   
..                ...       ...             ...      ...            ...   
995                 3        12               1        5           1736   
996                 1        30               1        2           3857   
997                 3        12               1        7            804   
998                 1        45               1        7           1845   
999                 0        45               0        2           4576   

     savings-account  employment-since  installment-rate  other-debtors  \
0                  4    

pandas.core.frame.DataFrame

**MCAR**

In [ ]:
generator_mcar10 = Inject_Missing_Values()
miss_mcar10,index_mcar10 = generator_mcar10.MCAR(X,missing_rate=10)
total_missing_percentage = miss_mcar10.isnull().sum().sum() / miss_mcar10.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar10  = pd.concat([miss_mcar10, Y], axis=1)
print(type(data_mcar10))
data_mcar10

Total Missing Percentage: 10.00%
<class 'pandas.core.frame.DataFrame'>


,checking-account,duration,credit-history,purpose,credit-amount,savings-account,employment-since,installment-rate,other-debtors,residence-since,...,other-installment,housing,existing-credits,job,number-people-provide-maintenance-for,telephone,foreign-worker,sex,marital-status,class-label
0,1.0,6.0,0.0,7.0,1169.0,4.0,3.0,4.0,2.0,4.0,...,NaN,1.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1
1,0.0,48.0,1.0,7.0,5951.0,2.0,0.0,2.0,2.0,2.0,...,1.0,1.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0
2,3.0,12.0,NaN,4.0,2096.0,2.0,1.0,2.0,NaN,3.0,...,1.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,0.0,1
3,1.0,42.0,1.0,5.0,7882.0,NaN,1.0,2.0,NaN,4.0,...,1.0,0.0,1.0,0.0,2.0,NaN,1.0,NaN,0.0,1
4,1.0,24.0,2.0,1.0,4870.0,2.0,0.0,3.0,NaN,4.0,...,1.0,0.0,2.0,0.0,NaN,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.0,12.0,1.0,5.0,1736.0,2.0,1.0,NaN,NaN,4.0,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,0.0,0.0,1
996,1.0,30.0,1.0,2.0,3857.0,2.0,0.0,4.0,2.0,4.0,...,1.0,NaN,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1
997,3.0,12.0,1.0,7.0,804.0,2.0,3.0,4.0,2.0,4.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1
998,1.0,45.0,1.0,7.0,1845.0,2.0,0.0,4.0,2.0,4.0,...,1.0,0.0,1.0,0.0,NaN,1.0,1.0,1.0,0.0,0


In [ ]:
generator15 = Inject_Missing_Values()
miss_mcar15,index_mcar15 = generator15.MCAR(X,missing_rate=15)
total_missing_percentage = miss_mcar15.isnull().sum().sum() / miss_mcar15.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar15 = df_combined = pd.concat([miss_mcar15, Y], axis=1)
print(type(data_mcar15))

Total Missing Percentage: 15.00%
<class 'pandas.core.frame.DataFrame'>


,checking-account,duration,credit-history,purpose,credit-amount,savings-account,employment-since,installment-rate,other-debtors,residence-since,...,other-installment,housing,existing-credits,job,number-people-provide-maintenance-for,telephone,foreign-worker,sex,marital-status,class-label
990,3.0,12.0,0.0,4.0,NaN,4.0,2.0,2.0,2.0,1.0,...,1.0,1.0,2.0,1.0,2.0,0.0,1.0,1.0,NaN,1
991,3.0,15.0,4.0,7.0,NaN,0.0,NaN,NaN,2.0,4.0,...,0.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,0.0,1
992,1.0,18.0,NaN,7.0,NaN,4.0,1.0,2.0,2.0,4.0,...,1.0,2.0,2.0,1.0,1.0,NaN,1.0,1.0,NaN,1
993,NaN,36.0,1.0,5.0,3959.0,2.0,NaN,4.0,NaN,3.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1
994,3.0,12.0,1.0,1.0,2390.0,4.0,3.0,4.0,2.0,3.0,...,1.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,0.0,1
995,NaN,NaN,1.0,5.0,1736.0,NaN,1.0,3.0,2.0,4.0,...,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0,0.0,1
996,NaN,30.0,1.0,2.0,3857.0,2.0,0.0,NaN,2.0,4.0,...,1.0,NaN,1.0,2.0,1.0,1.0,1.0,1.0,NaN,1
997,3.0,12.0,NaN,7.0,NaN,NaN,3.0,4.0,NaN,NaN,...,1.0,1.0,1.0,NaN,1.0,0.0,NaN,1.0,0.0,1
998,1.0,45.0,1.0,7.0,1845.0,2.0,0.0,4.0,2.0,4.0,...,1.0,0.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,0
999,NaN,45.0,0.0,2.0,4576.0,0.0,4.0,3.0,2.0,4.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   checking-account                       1000 non-null   int64
 1   duration                               1000 non-null   int64
 2   credit-history                         1000 non-null   int64
 3   purpose                                1000 non-null   int64
 4   credit-amount                          1000 non-null   int64
 5   savings-account                        1000 non-null   int64
 6   employment-since                       1000 non-null   int64
 7   installment-rate                       1000 non-null   int64
 8   other-debtors                          1000 non-null   int64
 9   residence-since                        1000 non-null   int64
 10  property                               1000 non-null   int64
 11  age                            

In [ ]:
generator20 = Inject_Missing_Values()
miss_mcar20,index_mcar20 = generator20.MCAR(X,missing_rate=20)
total_missing_percentage = miss_mcar20.isnull().sum().sum() / miss_mcar20.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar20 = pd.concat([miss_mcar20, Y], axis=1)



Total Missing Percentage: 20.00%


In [ ]:
generator25 = Inject_Missing_Values()
miss_mcar25,index_mcar25 = generator25.MCAR(X,missing_rate=25)
total_missing_percentage = miss_mcar25.isnull().sum().sum() / miss_mcar25.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar25 = pd.concat([miss_mcar25, Y], axis=1)


Total Missing Percentage: 25.00%


In [ ]:
generator30 = Inject_Missing_Values()
miss_mcar30,index_mcar30 = generator30.MCAR(X,missing_rate=30)
total_missing_percentage = miss_mcar30.isnull().sum().sum() / miss_mcar30.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar30 = pd.concat([miss_mcar30, Y], axis=1)

Total Missing Percentage: 30.00%


In [ ]:
generator35 = Inject_Missing_Values()
miss_mcar35,index_mcar35 = generator35.MCAR(X,missing_rate=35)
total_missing_percentage = miss_mcar35.isnull().sum().sum() / miss_mcar35.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar35 =  pd.concat([miss_mcar35, Y], axis=1)

Total Missing Percentage: 35.00%


In [ ]:
generator40 = Inject_Missing_Values()
miss_mcar40,index_mcar40 = generator40.MCAR(X,missing_rate=40)
total_missing_percentage = miss_mcar40.isnull().sum().sum() / miss_mcar40.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar40 = pd.concat([miss_mcar40, Y], axis=1)

Total Missing Percentage: 40.00%


In [27]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=45)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar45 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 45.00%


In [28]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=50)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar50 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 50.00%


In [29]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=55)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar55 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 55.00%


In [30]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=60)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar60 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 60.00%


In [31]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=65)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar65 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 65.00%


In [32]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=70)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar70 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 70.00%


In [33]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=75)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar75 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 75.00%


In [34]:
generator = Inject_Missing_Values()
miss_mcar,index_mcar = generator.MCAR(X,missing_rate=80)
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar80 = df_combined = pd.concat([miss_mcar, Y], axis=1)

Total Missing Percentage: 80.00%


In [35]:
generator_mcar85 = Inject_Missing_Values()
miss_mcar85,index_mcar = generator_mcar85.MCAR(X,missing_rate=85)
total_missing_percentage = miss_mcar85.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar85 = df_combined = pd.concat([miss_mcar85, Y], axis=1)

Total Missing Percentage: 85.00%


In [36]:
generator_mcar90 = Inject_Missing_Values()
miss_mcar90,index_mcar = generator_mcar90.MCAR(X,missing_rate=90)
total_missing_percentage = miss_mcar90.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
data_mcar90 = df_combined = pd.concat([miss_mcar90, Y], axis=1)

Total Missing Percentage: 90.00%


In [46]:
dependencies_mar = {"savings-account": {
        "influencers": ["age"],
        "condition": lambda row: True,  # Just a boolean condition
        "probability" : lambda row: 0.8 if row["age"]<=25 else 0.2 
    },
    "checking-account": {
        "influencers": ["age"],
        "condition": lambda row: True,  # Just a boolean condition
        "probability" : lambda row: 0.8 if row["age"]<=25 else 0.2 
    },
    "credit-amount": {
        "influencers": ["age"],
        "condition": lambda row: True,  # Just a boolean condition
        "probability" : lambda row: 0.8 if row["age"]<=25 else 0.2 
    },
    "employment-since": {
        "influencers": ["sex"],
        "condition": lambda row: True,  # Just a boolean condition
        "probability" : lambda row: 0.8 if row["sex"]==1 else 0.2 
    },
    "duration": {
        "influencers": ["sex"],
        "condition": lambda row: True,  # Just a boolean condition
        "probability" : lambda row: 0.8 if row["sex"]==1 else 0.2 
    }
}
dependencies_mnar = {
    "checking-account": {
        "influencers": ["checking-account"],
         "condition": lambda row: True,
        "probability": lambda row: 0.25 if row["checking-account"] == 0 else 0.05
    },
    "duration": {
        "influencers": ["duration"],
        "condition": lambda row: True,
        "probability": lambda row: 0.25 if row["duration"] <= 20 else 0.05
    },
    "savings-account": {
        "influencers": ["savings-account"],
        "condition": lambda row: True,
        "probability": lambda row: 0.2 if row["savings-account"] != 0 else 0.1
    },
    "employment-since": {
        "influencers": ["employment-since"],
         "condition": lambda row: True,
        "probability": lambda row: 0.2 if row["employment-since"] in [1,2] else 0.1
    },
    "credit-amount": {
        "influencers": ["credit-amount"],
         "condition": lambda row: True,
        "probability": lambda row: 0.25 if row["credit-amount"] > 5000 else 0.05
    }
}




generator_multi = Inject_Missing_Values()
miss_mcar_multi,index_mcar = generator_multi.MCAR(X,missing_rate=15)
miss_mar_multi,index_mar = generator_multi.MAR(miss_mcar_multi,dependencies_mar,missing_rate=15)
miss_mnar_multi,index_mnar = generator_multi.MNAR(miss_mar_multi,dependencies_mnar,missing_rate=15)

total_missing_percentage = miss_mnar_multi.isnull().sum().sum() / miss_mnar_multi.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")
miss_mnar_multi = pd.concat([miss_mnar_multi, Y], axis=1)


[(1, 5, 0.8), (10, 5, 0.8), (11, 5, 0.8), (12, 5, 0.8), (17, 5, 0.8), (35, 5, 0.8), (39, 5, 0.8), (43, 5, 0.8), (47, 5, 0.8), (58, 5, 0.8), (59, 5, 0.8), (67, 5, 0.8), (69, 5, 0.8), (79, 5, 0.8), (82, 5, 0.8), (93, 5, 0.8), (101, 5, 0.8), (102, 5, 0.8), (111, 5, 0.8), (113, 5, 0.8), (118, 5, 0.8), (120, 5, 0.8), (122, 5, 0.8), (130, 5, 0.8), (143, 5, 0.8), (144, 5, 0.8), (152, 5, 0.8), (155, 5, 0.8), (167, 5, 0.8), (168, 5, 0.8), (188, 5, 0.8), (203, 5, 0.8), (208, 5, 0.8), (217, 5, 0.8), (218, 5, 0.8), (221, 5, 0.8), (228, 5, 0.8), (229, 5, 0.8), (233, 5, 0.8), (242, 5, 0.8), (245, 5, 0.8), (249, 5, 0.8), (252, 5, 0.8), (254, 5, 0.8), (269, 5, 0.8), (280, 5, 0.8), (296, 5, 0.8), (306, 5, 0.8), (309, 5, 0.8), (311, 5, 0.8), (313, 5, 0.8), (332, 5, 0.8), (333, 5, 0.8), (334, 5, 0.8), (336, 5, 0.8), (337, 5, 0.8), (340, 5, 0.8), (342, 5, 0.8), (346, 5, 0.8), (347, 5, 0.8), (350, 5, 0.8), (355, 5, 0.8), (359, 5, 0.8), (363, 5, 0.8), (364, 5, 0.8), (367, 5, 0.8), (382, 5, 0.8), (385, 5, 0.

In [47]:
generator_mar30 = Inject_Missing_Values()
miss_mar30,index_mar30 = generator_mar30.MAR(X,dependencies_mar,missing_rate=30)


[(1, 5, 0.8), (10, 5, 0.8), (11, 5, 0.8), (12, 5, 0.8), (17, 5, 0.8), (35, 5, 0.8), (39, 5, 0.8), (43, 5, 0.8), (47, 5, 0.8), (52, 5, 0.8), (58, 5, 0.8), (59, 5, 0.8), (63, 5, 0.8), (67, 5, 0.8), (69, 5, 0.8), (79, 5, 0.8), (82, 5, 0.8), (93, 5, 0.8), (101, 5, 0.8), (102, 5, 0.8), (111, 5, 0.8), (113, 5, 0.8), (118, 5, 0.8), (120, 5, 0.8), (122, 5, 0.8), (130, 5, 0.8), (134, 5, 0.8), (143, 5, 0.8), (144, 5, 0.8), (152, 5, 0.8), (155, 5, 0.8), (167, 5, 0.8), (168, 5, 0.8), (188, 5, 0.8), (194, 5, 0.8), (203, 5, 0.8), (208, 5, 0.8), (217, 5, 0.8), (218, 5, 0.8), (221, 5, 0.8), (228, 5, 0.8), (229, 5, 0.8), (233, 5, 0.8), (236, 5, 0.8), (242, 5, 0.8), (245, 5, 0.8), (249, 5, 0.8), (252, 5, 0.8), (254, 5, 0.8), (258, 5, 0.8), (269, 5, 0.8), (280, 5, 0.8), (296, 5, 0.8), (306, 5, 0.8), (308, 5, 0.8), (309, 5, 0.8), (311, 5, 0.8), (313, 5, 0.8), (332, 5, 0.8), (333, 5, 0.8), (334, 5, 0.8), (336, 5, 0.8), (337, 5, 0.8), (340, 5, 0.8), (342, 5, 0.8), (346, 5, 0.8), (347, 5, 0.8), (350, 5, 0.8)

In [7]:
def conditional_mutual_information(data, X, Y, Z, delta=1):

  cmi = 0

  P_Z = data.groupby(Z).size()
  P_Z = P_Z / P_Z.sum()

  P_XZ = data.groupby(X + Z).size()
  P_XZ = P_XZ / P_XZ.sum()

  P_YZ = data.groupby(Y + Z).size()
  P_YZ = P_YZ / P_YZ.sum()

  P_XYZ = data.groupby(X + Y + Z).size()
  P_XYZ = P_XYZ / P_XYZ.sum()

  for ind in P_XYZ.index:
    x_ind = ind[:len(X)]
    y_ind = ind[len(X):len(X + Y)]
    z_ind = ind[len(X + Y):]

    xz_ind = x_ind + z_ind
    yz_ind = y_ind + z_ind
    xyz_ind = ind

    z_ind = pd.MultiIndex.from_tuples([z_ind], names=Z) if len(Z) != 1 else pd.Index(z_ind, name=Z[0])
    xz_ind = pd.MultiIndex.from_tuples([xz_ind], names=X + Z)
    yz_ind = pd.MultiIndex.from_tuples([yz_ind], names=Y + Z)
    xyz_ind = pd.MultiIndex.from_tuples([xyz_ind], names=X + Y + Z)

    cmi += delta * P_XYZ[xyz_ind].item() * np.log2(
      P_Z[z_ind].item() * P_XYZ[xyz_ind].item() / (P_XZ[xz_ind].item() * P_YZ[yz_ind].item()))

  return cmi

In [8]:
data = df # Convert tensor to numpy array
sensitive_attributes = [11, 19]

# Define different bin sizes for each sensitive attribute
bin_sizes = {
    11: 4,  # Binary attribute
    19: 2  # More categories
}


# Digitize multiple sensitive attributes with different bin sizes
#X_list = [
 #   np.digitize(data[:, col], bins=np.linspace(data[:, col].min(), data[:, col].max(), bin_sizes[col] + 1))
  #  for col in sensitive_attributes
#]
#x = np.column_stack(X_list)
#X = np.digitize(data[:,1],bins=np.linspace(data[:,1].min(), data[:,1].max(),1)) 
#y =np.digitize(Y,bins=np.linspace(Y.min(), Y.max(),4)) 
#z =  np.digitize(data[:, 7],bins=np.linspace(data[:, 7].min(), data[:, 7].max(), 4))

x = ['sex','age']
y = ['class-label']
z = ['savings-account', 'job']


cmi_value =conditional_mutual_information(df, x, y, z, delta=1)
print("Conditional Mutual Information:", cmi_value)

Conditional Mutual Information: 0.35071752538520246
